In [160]:
import numpy
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 




#train spider is training data
#train others is testing data

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\preni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [161]:
df=pd.read_json('train_spider.json')
df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
...,...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...",What are all the company names that have a boo...,"[What, are, all, the, company, names, that, ha...","{'from': {'table_units': [['table_unit', 2], [..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",Show the movie titles and book titles for all ...,"[Show, the, movie, titles, and, book, titles, ...","{'from': {'table_units': [['table_unit', 1], [..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",What are the titles of movies and books corres...,"[What, are, the, titles, of, movies, and, book...","{'from': {'table_units': [['table_unit', 1], [..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...",Show all company names with a movie directed i...,"[Show, all, company, names, with, a, movie, di...","{'from': {'table_units': [['table_unit', 1], [..."


In [162]:
df_sample= df.iloc[:-5999,:].copy()
df_sample

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
...,...,...,...,...,...,...,...
996,university_basketball,SELECT avg(enrollment) FROM university WHERE f...,"[SELECT, avg, (, enrollment, ), FROM, universi...","[select, avg, (, enrollment, ), from, universi...",Return the average enrollment of universities ...,"[Return, the, average, enrollment, of, univers...","{'from': {'table_units': [['table_unit', 1]], ..."
997,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",Show the enrollment and primary_conference of ...,"[Show, the, enrollment, and, primary_conferenc...","{'from': {'table_units': [['table_unit', 1]], ..."
998,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",What are the enrollment and primary conference...,"[What, are, the, enrollment, and, primary, con...","{'from': {'table_units': [['table_unit', 1]], ..."
999,university_basketball,"SELECT sum(enrollment) , min(enrollment) FROM...","[SELECT, sum, (, enrollment, ), ,, min, (, enr...","[select, sum, (, enrollment, ), ,, min, (, enr...",What is the total and minimum enrollment of al...,"[What, is, the, total, and, minimum, enrollmen...","{'from': {'table_units': [['table_unit', 1]], ..."


In [163]:
###preprocessing

In [164]:
#convert question coloumn into lower case
df_sample.question = df_sample.question.astype(str).str.lower()
df_sample

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",how many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","list the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","list the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",what are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",what is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
...,...,...,...,...,...,...,...
996,university_basketball,SELECT avg(enrollment) FROM university WHERE f...,"[SELECT, avg, (, enrollment, ), FROM, universi...","[select, avg, (, enrollment, ), from, universi...",return the average enrollment of universities ...,"[Return, the, average, enrollment, of, univers...","{'from': {'table_units': [['table_unit', 1]], ..."
997,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",show the enrollment and primary_conference of ...,"[Show, the, enrollment, and, primary_conferenc...","{'from': {'table_units': [['table_unit', 1]], ..."
998,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",what are the enrollment and primary conference...,"[What, are, the, enrollment, and, primary, con...","{'from': {'table_units': [['table_unit', 1]], ..."
999,university_basketball,"SELECT sum(enrollment) , min(enrollment) FROM...","[SELECT, sum, (, enrollment, ), ,, min, (, enr...","[select, sum, (, enrollment, ), ,, min, (, enr...",what is the total and minimum enrollment of al...,"[What, is, the, total, and, minimum, enrollmen...","{'from': {'table_units': [['table_unit', 1]], ..."


In [165]:
#removing punctuations 
df_sample["Question_NoPunctuation"] = df_sample['question'].str.replace('[^\w\s]','')
df_sample

<ipython-input-165-765c902a064f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_sample["Question_NoPunctuation"] = df_sample['question'].str.replace('[^\w\s]','')


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,Question_NoPunctuation
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",how many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ...",how many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","list the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ...",list the name born state and age of the heads ...
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","list the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ...",list the creation year name and budget of each...
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",what are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ...",what are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",what is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ...",what is the average number of employees of the...
...,...,...,...,...,...,...,...,...
996,university_basketball,SELECT avg(enrollment) FROM university WHERE f...,"[SELECT, avg, (, enrollment, ), FROM, universi...","[select, avg, (, enrollment, ), from, universi...",return the average enrollment of universities ...,"[Return, the, average, enrollment, of, univers...","{'from': {'table_units': [['table_unit', 1]], ...",return the average enrollment of universities ...
997,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",show the enrollment and primary_conference of ...,"[Show, the, enrollment, and, primary_conferenc...","{'from': {'table_units': [['table_unit', 1]], ...",show the enrollment and primary_conference of ...
998,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",what are the enrollment and primary conference...,"[What, are, the, enrollment, and, primary, con...","{'from': {'table_units': [['table_unit', 1]], ...",what are the enrollment and primary conference...
999,university_basketball,"SELECT sum(enrollment) , min(enrollment) FROM...","[SELECT, sum, (, enrollment, ), ,, min, (, enr...","[select, sum, (, enrollment, ), ,, min, (, enr...",what is the total and minimum enrollment of al...,"[What, is, the, total, and, minimum, enrollmen...","{'from': {'table_units': [['table_unit', 1]], ...",what is the total and minimum enrollment of al...


In [166]:
#tokenizing the unpunctuated coloumn
nltk.download('punkt')
index=0
df_sample["Question_NoPunc_tokened"]= df_sample['Question_NoPunctuation']
for i in df_sample['Question_NoPunctuation']:
    df_sample["Question_NoPunc_tokened"][index] = word_tokenize(i)
    index=index+1
df_sample

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\preni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,Question_NoPunctuation,Question_NoPunc_tokened
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",how many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ...",how many heads of the departments are older th...,"[how, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","list the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ...",list the name born state and age of the heads ...,"[list, the, name, born, state, and, age, of, t..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","list the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ...",list the creation year name and budget of each...,"[list, the, creation, year, name, and, budget,..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",what are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ...",what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",what is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ...",what is the average number of employees of the...,"[what, is, the, average, number, of, employees..."
...,...,...,...,...,...,...,...,...,...
996,university_basketball,SELECT avg(enrollment) FROM university WHERE f...,"[SELECT, avg, (, enrollment, ), FROM, universi...","[select, avg, (, enrollment, ), from, universi...",return the average enrollment of universities ...,"[Return, the, average, enrollment, of, univers...","{'from': {'table_units': [['table_unit', 1]], ...",return the average enrollment of universities ...,"[return, the, average, enrollment, of, univers..."
997,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",show the enrollment and primary_conference of ...,"[Show, the, enrollment, and, primary_conferenc...","{'from': {'table_units': [['table_unit', 1]], ...",show the enrollment and primary_conference of ...,"[show, the, enrollment, and, primary_conferenc..."
998,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",what are the enrollment and primary conference...,"[What, are, the, enrollment, and, primary, con...","{'from': {'table_units': [['table_unit', 1]], ...",what are the enrollment and primary conference...,"[what, are, the, enrollment, and, primary, con..."
999,university_basketball,"SELECT sum(enrollment) , min(enrollment) FROM...","[SELECT, sum, (, enrollment, ), ,, min, (, enr...","[select, sum, (, enrollment, ), ,, min, (, enr...",what is the total and minimum enrollment of al...,"[What, is, the, total, and, minimum, enrollmen...","{'from': {'table_units': [['table_unit', 1]], ...",what is the total and minimum e

In [173]:
#lemmetization
index=0
df_sample["lemmetized_text"]= df_sample['Question_NoPunc_tokened']
lemmatizer = WordNetLemmatizer()
k=0
for index in df_sample['Question_NoPunc_tokened']: 
    df_sample["lemmetized_text"][k] = ' '.join([lemmatizer.lemmatize(w) for w in index])
    k=k+1
df_sample

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,Question_NoPunctuation,Question_NoPunc_tokened,lemmetized_text
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",how many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ...",how many heads of the departments are older th...,"[how, many, heads, of, the, departments, are, ...",how many head of the department are older than 56
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","list the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ...",list the name born state and age of the heads ...,"[list, the, name, born, state, and, age, of, t...",list the name born state and age of the head o...
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","list the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ...",list the creation year name and budget of each...,"[list, the, creation, year, name, and, budget,...",list the creation year name and budget of each...
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",what are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ...",what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget...",what are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",what is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ...",what is the average number of employees of the...,"[what, is, the, average, number, of, employees...",what is the average number of employee of the ...
...,...,...,...,...,...,...,...,...,...,...
996,university_basketball,SELECT avg(enrollment) FROM university WHERE f...,"[SELECT, avg, (, enrollment, ), FROM, universi...","[select, avg, (, enrollment, ), from, universi...",return the average enrollment of universities ...,"[Return, the, average, enrollment, of, univers...","{'from': {'table_units': [['table_unit', 1]], ...",return the average enrollment of universities ...,"[return, the, average, enrollment, of, univers...",return the average enrollment of university fo...
997,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",show the enrollment and primary_conference of ...,"[Show, the, enrollment, and, primary_conferenc...","{'from': {'table_units': [['table_unit', 1]], ...",show the enrollment and primary_conference of ...,"[show, the, enrollment, and, primary_conferenc...",show the enrollment and primary_conference of ...
998,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",what are the enrollment and primary conference...,"[What, are, the, enrollment, and, primary, con...","{'from': {'table_units': [['table_unit', 1]], ...",what are the enrollment and primary conference...,"[what, are, the, enrollment, and, primary, con.

In [175]:
#pos tagging
nltk.download('averaged_perceptron_tagger')
index=0
df_sample["pos_tagged"]= df_sample['Question_NoPunc_tokened']
lemmatizer = WordNetLemmatizer()
k=0
for index in df_sample['Question_NoPunc_tokened']: 
    df_sample["pos_tagged"][k] = nltk.pos_tag(index)
    k=k+1
df_sample

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\preni\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,Question_NoPunctuation,Question_NoPunc_tokened,lemmetized_text,pos_tagged
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",how many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ...",how many heads of the departments are older th...,"[how, many, heads, of, the, departments, are, ...",how many head of the department are older than 56,"[(how, WRB), (many, JJ), (heads, NNS), (of, IN..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","list the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ...",list the name born state and age of the heads ...,"[list, the, name, born, state, and, age, of, t...",list the name born state and age of the head o...,"[(list, NN), (the, DT), (name, NN), (born, JJ)..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","list the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ...",list the creation year name and budget of each...,"[list, the, creation, year, name, and, budget,...",list the creation year name and budget of each...,"[(list, NN), (the, DT), (creation, NN), (year,..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",what are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ...",what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget...",what are the maximum and minimum budget of the...,"[(what, WDT), (are, VBP), (the, DT), (maximum,..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",what is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ...",what is the average number of employees of the...,"[what, is, the, average, number, of, employees...",what is the average number of employee of the ...,"[(what, WP), (is, VBZ), (the, DT), (average, J..."
...,...,...,...,...,...,...,...,...,...,...,...
996,university_basketball,SELECT avg(enrollment) FROM university WHERE f...,"[SELECT, avg, (, enrollment, ), FROM, universi...","[select, avg, (, enrollment, ), from, universi...",return the average enrollment of universities ...,"[Return, the, average, enrollment, of, univers...","{'from': {'table_units': [['table_unit', 1]], ...",return the average enrollment of universities ...,"[return, the, average, enrollment, of, univers...",return the average enrollment of university fo...,"[(return, VB), (the, DT), (average, JJ), (enro..."
997,university_basketball,"SELECT enrollment , primary_conference FROM u...","[SELECT, enrollment, ,, primary_conference, FR...","[select, enrollment, ,, primary_conference, fr...",show the enrollment and primary_conference of ...,"[Show, the, enrollment, and, primary_conferenc...","{'from': {'table_units': [['table_unit', 1]], ...",show the enrollment and primary_conference of ...,"[show, the, enrollment, and, primary_conferenc...",show the enrollment and primary_conference of ...,"[(show, VB), (the, DT), (enrollment, NN), (and..."
998,university_basketball,"SELECT enrollment , primary